In [49]:
%matplotlib inline
import importlib
import codecs
import os
import pandas
import numpy 
import datetime as dt
import scipy.stats
import pandasql.sqldf
import matplotlib.axis as axis
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pandas.options.display.max_rows = 200
pandas.options.display.max_columns = 50
pandas.options.display.width = 300
pandas.set_option('display.float_format', lambda x: '%.8f' % x)
import trm3
import trm3.impute
import trm3.utility

input_file = r"D:\projects\Python36-Package-Development\trm3pylib\data-csv\zillow_train.csv"
with codecs.open(input_file, 'r', encoding='utf-8') as f:
   df= pandas.read_csv(f, sep=",", header=0, low_memory=False, parse_dates=True)

df["datecol"] = pandas.to_datetime(df.transactiondate)
df["interval"] = df["datecol"] - dt.datetime.now().date()
df["fips"].fillna(value=df["fips"].mode().tolist()[0], inplace=True)


In [66]:
importlib.reload(trm3)
importlib.reload(trm3.impute)
importlib.reload(trm3.utility)

<module 'trm3' from 'D:\\projects\\Python36-Package-Development\\trm3pylib\\trm3\\__init__.py'>

<module 'trm3.impute' from 'D:\\projects\\Python36-Package-Development\\trm3pylib\\trm3\\impute.py'>

<module 'trm3.utility' from 'D:\\projects\\Python36-Package-Development\\trm3pylib\\trm3\\utility.py'>

In [70]:
trm3.utility.get_size(df, unit="MB")

# x = trm3.utility.MemoryUnits.GB
# x.abbreviation

144.859875 Megabytes (MB)


144.859875

In [ ]:
def autolabel(rects, ax, percentages=False):
    # Get y-axis height to calculate label position from.
    (y_bottom, y_top) = ax.get_ylim()
    y_height = y_top - y_bottom

    for rect in rects:
        height = rect.get_height()

        # Fraction of axis height taken up by this rectangle
        p_height = (height / y_height)

        # If we can fit the label above the column, do that;
        # otherwise, put it inside the column.
        if p_height > 0.95: # arbitrary; 95% looked good to me.
            label_position = height - (y_height * 0.05)
        else:
            label_position = height + (y_height * 0.01)
        
        height = 100*height
        ax.text(rect.get_x() + rect.get_width()/2., label_position,
                '%.1f %%' % height,
                ha='center', va='bottom')
 

In [ ]:
col="calculatedfinishedsquarefeet"
plt.rcParams["figure.figsize"] = [12, 9]
nomiss = df[col].notnull()
histogram = plt.figure()
weights = numpy.ones_like(df.loc[nomiss,col])/float(len(df.loc[nomiss,col]))
n, bins, patches = plt.hist(df.loc[nomiss,col], alpha=.5, facecolor='green', 
                            edgecolor='black', linewidth=1.2, weights=weights)
plt.xlabel(col)
plt.ylabel('# of Observations')
autolabel(patches, plt.gca(), percentages=True)
plt.show()
plt.close()


In [ ]:
df.bedroomcnt.hist()

In [ ]:
importlib.reload(trm3.impute)    
my_imputer = trm3.impute.Imputer(train_df = df,
                                 test_df = None,
                                 columns = [x for x in df.columns.tolist() if x[0:3] != "dv_" and x !="fips"],
#                                  columns = ["datecol"],
                                 groupby = ["fips"],
                                 label = "dv_avg_error",
                                 method = 2,
                                 mode_fallback_distinct_values = 3,
                                 mean_label_min_obs = .005,
                                 mean_label_bins = 10,
                                 nominal_new_category_min_obs = .01,
                                 missing_indicators = False,
                                 missing_indicators_min_obs = .01,
                                 model_x_columns = None,
                                 verbose=False
                                )

my_imputer_rules = my_imputer.fit()
my_imputer_rules.impute(impute_df=df, inplace=False, verbose=True)

In [ ]:
print(my_imputer_rules.python_code())

In [59]:
'{0:,}'.format(2999887766)

'2,999,887,766'

TypeError: unsupported operand type(s) for /: 'int' and 'set'

In [ ]:
cols = df.columns.tolist()
cols.sort()

In [ ]:
sdf = trm3.utility.df_contents(df, display_html_output=True)
#                   excel_filename=r"D:\projects\Python36-Package-Development\temp\test01.xlsx",
#                   excel_sheet_name="DF Summary")